In [1]:
#Import modules:
import sys
import os, errno
import user_interface as ui
import matplotlib.pyplot as plt
import iris
import iris.quickplot as iplt
import numpy as np



from __future__ import (absolute_import, division, print_function)
from six.moves import (filter, input, map, range, zip)  # noqa



In [2]:
cubes = iris.load('../../coads_climatology.nc.nc')

In [3]:
print(cubes)

0: SEA SURFACE TEMPERATURE / (degree_celsius) (TIME: 12; latitude: 90; longitude: 180)
1: AIR TEMPERATURE / (degree_celsius)  (TIME: 12; latitude: 90; longitude: 180)
2: ZONAL WIND / (meter per second)     (TIME: 12; latitude: 90; longitude: 180)
3: MERIDIONAL WIND / (meter per second) (TIME: 12; latitude: 90; longitude: 180)


In [4]:
for c in cubes:
    if c.name() == 'AIR TEMPERATURE':
        c0 = c
print(c0)


AIR TEMPERATURE / (degree_celsius)  (TIME: 12; latitude: 90; longitude: 180)
     Dimension coordinates:
          TIME                           x             -              -
          latitude                       -             x              -
          longitude                      -             -              x
     Attributes:
          DODS_EXTRA.Unlimited_Dimension: TIME
          NC_GLOBAL.history: FERRET V4.30 (debug/no GUI) 15-Aug-96
          history: From coads_climatology


In [5]:
for coord in c0.coords():
    print(coord.name())

TIME
latitude
longitude


In [6]:
print(c0.coord('latitude'))
print(c0.coord('longitude'))

DimCoord(array([-89., -87., -85., -83., -81., -79., -77., -75., -73., -71., -69.,
       -67., -65., -63., -61., -59., -57., -55., -53., -51., -49., -47.,
       -45., -43., -41., -39., -37., -35., -33., -31., -29., -27., -25.,
       -23., -21., -19., -17., -15., -13., -11.,  -9.,  -7.,  -5.,  -3.,
        -1.,   1.,   3.,   5.,   7.,   9.,  11.,  13.,  15.,  17.,  19.,
        21.,  23.,  25.,  27.,  29.,  31.,  33.,  35.,  37.,  39.,  41.,
        43.,  45.,  47.,  49.,  51.,  53.,  55.,  57.,  59.,  61.,  63.,
        65.,  67.,  69.,  71.,  73.,  75.,  77.,  79.,  81.,  83.,  85.,
        87.,  89.]), standard_name='latitude', units=Unit('degrees'), var_name='COADSY', attributes={'point_spacing': 'even'})
DimCoord(array([  21.,   23.,   25.,   27.,   29.,   31.,   33.,   35.,   37.,
         39.,   41.,   43.,   45.,   47.,   49.,   51.,   53.,   55.,
         57.,   59.,   61.,   63.,   65.,   67.,   69.,   71.,   73.,
         75.,   77.,   79.,   81.,   83.,   85.,   87.,   89.

In [7]:
#This will give error:
#ValueError: 
#zero not allowed as a reference year, does not exist in Julian or Gregorian calendars

if(0): print(c0.coord('TIME'))

#Why? ...See the error and next box for the solution.

In [8]:
#Reason for the error above: The reference year is set to zero...
#See...
print(c0.coord('TIME').units)

hour since 0000-01-01 00:00:00


In [9]:
#We change the reference year by doing the following:
c0.coord('TIME').units='hour since epoch'
c0t = c0.coord('TIME')
#"epoch" = 1st of january of 1970.
print(c0t.units)


hour since 1970-01-01 00:00:00


In [10]:
#Now we can safely print the coordinate:
print(c0.coord('TIME'))

DimCoord([1970-01-16 06:00:00, 1970-02-15 16:29:06.000004,
       1970-03-18 02:58:12, 1970-04-17 13:27:18, 1970-05-17 23:56:24,
       1970-06-17 10:25:30.000009, 1970-07-17 20:54:36,
       1970-08-17 07:23:42, 1970-09-16 17:52:48.000003,
       1970-10-17 04:21:54, 1970-11-16 14:51:00, 1970-12-17 01:20:06], standard_name=None, calendar='gregorian', var_name='TIME', attributes={'time_origin': '1-JAN-0000 00:00:00', 'modulo': ' '})


In [11]:
#As the 'time_origin' attribute is still wrong...
print(c0.coord('TIME').attributes['time_origin'])

#We change it by:
c0.coord('TIME').attributes['time_origin'] = '01-JAN-1970 00:00:00'

print(c0.coord('TIME').attributes['time_origin'])

1-JAN-0000 00:00:00
01-JAN-1970 00:00:00


In [12]:
#We need to extract a 1D cube and for this we create constraints:
print("The original cube:\n {}\n".format(c0))
c1 = c0.extract(iris.Constraint(longitude=45))
print("Extracting a sub-cube with only longitude = 45:\n {}\n".format(c1))
c2 = c1.extract(iris.Constraint(latitude=47))
print("Extracting again a sub-cube with only latitude = 47:\n {}\n".format(c2))

#There's another way of extacting a subset of a cube:
#Another way of slicing a cube:
c3 = c0[:,45,80]
print("Extracting at once by indices:\n {}\n".format(c3))

The original cube:
 AIR TEMPERATURE / (degree_celsius)  (TIME: 12; latitude: 90; longitude: 180)
     Dimension coordinates:
          TIME                           x             -              -
          latitude                       -             x              -
          longitude                      -             -              x
     Attributes:
          DODS_EXTRA.Unlimited_Dimension: TIME
          NC_GLOBAL.history: FERRET V4.30 (debug/no GUI) 15-Aug-96
          history: From coads_climatology

Extracting a sub-cube with only longitude = 45:
 AIR TEMPERATURE / (degree_celsius)  (TIME: 12; latitude: 90)
     Dimension coordinates:
          TIME                           x             -
          latitude                       -             x
     Scalar coordinates:
          longitude: 45.0 degrees
     Attributes:
          DODS_EXTRA.Unlimited_Dimension: TIME
          NC_GLOBAL.history: FERRET V4.30 (debug/no GUI) 15-Aug-96
          history: From coads_climatology



In [13]:
#If we print c2.data we get something weird:
if(1): print(c2.data)

#That is because cube thata is a "masked array" object.
print(type(c0.data))

#To show only the valid entries, we use:
print((c0.data.compressed()))

#How many entries are not masked, i.e. valid?
print(c0.data.size)
print(c0.data.compressed().size)

[-- -- -- -- -- -- -- -- -- -- -- --]
<class 'numpy.ma.core.MaskedArray'>
[ -1.17999995  -1.45724142  -1.76909089 ..., -37.43999863 -39.85000229
 -23.67000008]
194400
107194


In [14]:
#That was not a good way of getting a 1D cube. We better take the average.
#Indeed, constraining the cube to just one cell I don't think makes much sense...

#1. We put bound to the cells in order to take an area-average:
if not c0.coord('longitude').has_bounds():
    c0.coord('longitude').guess_bounds()

if not c0.coord('latitude').has_bounds():
    c0.coord('latitude').guess_bounds()

#2. We get the area weights of the cells composing the region:
grid_areas = iris.analysis.cartography.area_weights(c0)

#3. We "collapse" our 2D+Time cube into a 0D+Time by averaging using MEAN aggregator:
c0collapsed = c0.collapsed(['longitude', 'latitude'], iris.analysis.MEAN, weights=grid_areas)

print(c0collapsed)

AIR TEMPERATURE / (degree_celsius)  (TIME: 12)
     Dimension coordinates:
          TIME                           x
     Scalar coordinates:
          latitude: 0.0 degrees, bound=(-90.0, 90.0) degrees
          longitude: 200.0 degrees, bound=(20.0, 380.0) degrees
     Attributes:
          DODS_EXTRA.Unlimited_Dimension: TIME
          NC_GLOBAL.history: FERRET V4.30 (debug/no GUI) 15-Aug-96
          history: From coads_climatology
     Cell methods:
          mean: longitude, latitude


../lib/iris/analysis/cartography.py:327: UserWarning: Using DEFAULT_SPHERICAL_EARTH_RADIUS.
  warnings.warn("Using DEFAULT_SPHERICAL_EARTH_RADIUS.")


In [15]:
#We now want to interpolate between tmin and tmax. 

#We get the bounds:
if not c0.coord('TIME').has_bounds():
    c0.coord('TIME').guess_bounds()
minT = c0.coord('TIME').bounds.min()
maxT = c0.coord('TIME').bounds.max()

#We set the sample points on the coordinate TIME:
sample_points = [('TIME',np.linspace(minT,maxT))]

#We make a 'Linear' interpolation. The other option is 'Nearest'
c_interp = c0collapsed.interpolate(sample_points, iris.analysis.Linear())

#Notice that the extracted and the interpolated cubes look the same
#except for the number of time points available (12 the original, vs 50 the interpolated)
print("Original collapsed Cube: \n{}\n".format(c0collapsed))
print("Interpolated Cube: \n{}\n".format(c_interp))

Original collapsed Cube: 
AIR TEMPERATURE / (degree_celsius)  (TIME: 12)
     Dimension coordinates:
          TIME                           x
     Scalar coordinates:
          latitude: 0.0 degrees, bound=(-90.0, 90.0) degrees
          longitude: 200.0 degrees, bound=(20.0, 380.0) degrees
     Attributes:
          DODS_EXTRA.Unlimited_Dimension: TIME
          NC_GLOBAL.history: FERRET V4.30 (debug/no GUI) 15-Aug-96
          history: From coads_climatology
     Cell methods:
          mean: longitude, latitude

Interpolated Cube: 
AIR TEMPERATURE / (degree_celsius)  (TIME: 50)
     Dimension coordinates:
          TIME                           x
     Scalar coordinates:
          latitude: 0.0 degrees, bound=(-90.0, 90.0) degrees
          longitude: 200.0 degrees, bound=(20.0, 380.0) degrees
     Attributes:
          DODS_EXTRA.Unlimited_Dimension: TIME
          NC_GLOBAL.history: FERRET V4.30 (debug/no GUI) 15-Aug-96
          history: From coads_climatology
     Cell method

In [16]:
#We now plot the evolution of the temperature:
iplt.plot(c0collapsed,'ro')
iplt.plot(c_interp,'b.')
plt.show()

In [17]:
#We now do the reverse...

#We get the bounds:
if not c0.coord('longitude').has_bounds():
    c0.coord('longitude').guess_bounds()
minLon = c0.coord('longitude').bounds.min()
maxLon = c0.coord('longitude').bounds.max()

if not c0.coord('latitude').has_bounds():
    c0.coord('latitude').guess_bounds()
minLat = c0.coord('latitude').bounds.min()
maxLat = c0.coord('latitude').bounds.max()

#We set the sample points on the coordinate TIME:
sample_points = [('longitude',np.linspace(minLon,maxLon)),('latitude',np.linspace(minLat,maxLat))]

#We make a 'Linear' interpolation. The other option is 'Nearest'
c_interpol = c0.interpolate(sample_points, iris.analysis.Linear())
#c_interpol2 = c_interpol.interpolate([('longitude',[45]),('latitude',[47])],iris.analysis.Linear())
#Notice that the extracted and the interpolated cubes look the same
#except for the number of time points available (12 the original, vs 50 the interpolated)
#print("Original collapsed Cube: \n{}\n".format(c0collapsed))
print("Interpolated Cube: \n{}\n".format(c_interpol))
#print("Interpolated Cube2: \n{}\n".format(c_interpol2))

Interpolated Cube: 
AIR TEMPERATURE / (degree_celsius)  (TIME: 12; latitude: 50; longitude: 50)
     Dimension coordinates:
          TIME                           x             -              -
          latitude                       -             x              -
          longitude                      -             -              x
     Attributes:
          DODS_EXTRA.Unlimited_Dimension: TIME
          NC_GLOBAL.history: FERRET V4.30 (debug/no GUI) 15-Aug-96
          history: From coads_climatology



In [18]:
print(c0)

AIR TEMPERATURE / (degree_celsius)  (TIME: 12; latitude: 90; longitude: 180)
     Dimension coordinates:
          TIME                           x             -              -
          latitude                       -             x              -
          longitude                      -             -              x
     Attributes:
          DODS_EXTRA.Unlimited_Dimension: TIME
          NC_GLOBAL.history: FERRET V4.30 (debug/no GUI) 15-Aug-96
          history: From coads_climatology
